In [7]:
# 남은 할일 
# 기사 제목 끌고 와서 라벨링 해준다
# 그 데이터 가지고 정확도를 예측한다. 이 때 사용될 건 accuracy, f1 score 

import pandas as pd
from konlpy.tag import Hannanum  # Hannanum 형태소 분석기 불러오기
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

search = input("회사명(csv 파일명) : ")
num = int(input("몇 번째 테스트? "))
# csv
df = pd.read_csv(f'csv/{search} 뉴스원문{num}.csv')

# KoELECTRA 모델 로드
model_name = "hyunwoongko/kobart"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 형태소 분석 함수
hannanum = Hannanum()  # Hannanum 형태소 분석기 객체 생성
def tokenize(text):
    return hannanum.morphs(text)

# 이거 수정하고 결국에는 라벨링이 필요하다......

# df['content'] = df['title']

# content 열에 대해 형태소 분석 적용
df['title'] = df['title'].apply(tokenize)

# 감성 분석을 위한 전처리 함수
def preprocess(text):
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    inputs.to(device)
    return inputs

# 예측 함수
def predict(inputs):
    outputs = model(**inputs)
    logits = outputs.logits
    probs = logits.softmax(dim=-1)
    return probs[0].detach().cpu().numpy()

# content 열에 대해 예측 수행
df['sentiment'] = df['title'].apply(lambda x: predict(preprocess(' '.join(x))))

# 감성 분석 결과를 문자 형식으로 변환
# 조금 더 디테일하게 4.7 이하 부정, 5.3 이상 긍정, 그 외 중립 추가
def convert_sentiment(probs):
    if probs[0] < 0.5:
        return 0
    elif probs[0]>= 0.5:
        return 1
    else:
        return '중립'

df['sentiment'] = df['sentiment'].apply(convert_sentiment)

회사명(csv 파일명) : 삼성전자
몇 번째 테스트? 15


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
Some weights of BartForSequenceClassification were not initialized from the model checkpoint at hyunwoongko/kobart and are newly initialized: ['classification_head.dense.weight', 'classification_head.dense.bias', 'classification_head.out_proj.weight', 'classification_head.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<class 'list'>


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [8]:
# csv 파일로 저장
df.to_csv(f'csv/{search}_주가뉴스(KoBart season2){num}.csv', index=False)

# 결과 확인
print(df['title'])
print(df['sentiment'])

0     [삼성전자, ,, 갤럭시·노트북·TV서, ', 자, 가, 수, 이, 리, ', 국내...
1        [[, 특징주, ], 삼성전자·SK하이닉스,, 52주, 신, 고, 가, 아, 기록]
2           [작년, 주주, 배당액, 1위, 는, 삼성전자…총, 배당금, 은, 6.7%↓]
3          [갤S22·갤북, 스스로, 고친다…삼성전자, ‘자, 가, 수리’, 국내, 도입]
4            [", 청년, 에, 일경험"…SK하이닉스·삼성전자, 등, 2100명, 모집]
5              [삼성전자, 갤럭시, 제품, 직접, 수리, 가능, 하, 어, 지, ㄴ다]
6     [", 이번, 엔, 중동"…삼성전자,, 두바, 이, 리조트, 서, 2023년형, T...
7            [삼성전자, ,, 두바이, 서, 2023년형, 네오QLED, 신제품, 선봬]
8     [", 이제, 집, 에서, 고치, 세, 요, ", ..., 삼성전자, ,, ', 자...
9       ["머스크, 불, 만, 터트리, ㄹ, 지경인데"…삼성전자·하이닉스, 웃, 는, 이유]
10               [삼성전자, 신, 고, 가, 아, 행진…"8만전자, 갈, ㄴ다, "]
11       [", 삼성전자, ,, 책임경영, 필요성↑…이재용, 등기이사, 복귀, 가능성, "]
12          [삼성전자, ,, 외국, 이, ㄴ, 폭풍매수, 에, '7만전자, 굳히기, ']
13       [삼성전자, ,, 7.1만원, 진입…반도체株, 훈풍, 지, 속, [, 특징주, ]]
14    [", 삼성전자, 가, 또, ?"..., 신제품, 에, 적용, 하, ㄴ, ', 첨단...
15    [[, 특징주, ], 날개, 단, 삼성전자‧SK하이닉스…또, 52주, 최고, 가, 경신]
16    [돈다발, 들, 고, 달리, 어, 오, ㄴ, 외국인들…“삼성전자, 사, 고, 싶습니다”]
17    [신, 고, 가, 아, 행진, 삼성전자·SK하이닉스…"반도체, 질주, 는, 